# Dự đoán giá chứng khoán trong 1 ngày tiếp theo của Amazon
---

## 1. Import thư viện
---

In [232]:
import pandas as pd 

# phải cài cái này để load dữ liệu về


import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPRegressor, MLPClassifier

from sklearn.linear_model import LogisticRegression
# vẽ biểu đồ cho ngầu 
import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# xử lý thời gian 
from datetime import datetime

# keras for LSTM
# phải cài cái này 
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, LSTM

## 2. READ DATA
---

In [555]:
df = pd.read_csv('BTCUSDT_15MinuteBars.csv')

df
# df = pd.read_csv('AMZN.csv')
# df = df.set_index('Date')
# df

timestamp      open      high       low     close  \
0      2019-05-23 00:00:00   7627.80   7650.00   7571.18   7638.42   
1      2019-05-23 00:15:00   7635.96   7638.54   7538.81   7564.38   
2      2019-05-23 00:30:00   7562.70   7638.38   7533.00   7634.62   
3      2019-05-23 00:45:00   7638.14   7648.24   7610.00   7636.28   
4      2019-05-23 01:00:00   7636.30   7681.70   7634.01   7655.97   
...                    ...       ...       ...       ...       ...   
64892  2021-03-30 11:30:00  58990.66  59044.86  58884.64  59024.10   
64893  2021-03-30 11:45:00  59024.11  59246.93  58920.46  59240.34   
64894  2021-03-30 12:00:00  59240.33  59368.00  59024.15  59196.50   
64895  2021-03-30 12:15:00  59196.50  59200.00  59015.32  59059.30   
64896  2021-03-30 12:30:00  59059.31  59199.00  59050.00  59055.01   

            volume     close_time      quote_av  trades  tb_base_av  \
0       901.924709  1558570499999  6.869673e+06    7447  462.944261   
1       959.017267  1558571399999  7.275112e+06    7295  498.544897   
2       849.260997  1558572299999  6.449506e+06    6300  407.117683   
3       611.505913  1558573199999  4.667999e+06    4547  217.545582   
4       460.455221  1558574099999  3.527268e+06    5243  247.065081   
...            ...            ...           ...     ...         ...   
64892   486.970642  1617104699999  2.872185e+07   17747  212.085032   
64893   993.251443  1617105599999  5.870948e+07   20432  647.303097   
64894  1130.305391  1617106499999  6.696778e+07   26825  599.128578   
64895   563.273870  1617107399999  3.328597e+07   17446  306.211084   
64896   243.102030  1617108299999  1.437641e+07    9033  123.750650   

        tb_quote_av  ignore  
0      3.526314e+06       0  
1      3.783333e+06       0  
2      3.092928e+06       0  
3      1.660150e+06       0  
4      1.892431e+06       0  
...             ...     ...  
64892  1.250893e+07       0  
64893  3.825925e+07       0  
64894  3.550147e+07       0  
64895  1.809445e+07       0  
64896  7.317754e+06       0  

[64897 rows x 12 columns]

cộng , trừ => nến

In [556]:
def Create_Train(df,n,percent):

        df.drop('timestamp', inplace = True, axis = 1 )
        df.drop('close_time', inplace = True, axis = 1 )
        df.drop('quote_av', inplace = True, axis = 1 )
        df.drop('trades', inplace = True, axis = 1 )
        df.drop('tb_base_av', inplace = True, axis = 1 )
        df.drop('tb_quote_av', inplace = True, axis = 1 )
        df.drop('ignore', inplace = True, axis = 1 )
        df.drop('volume', inplace = True, axis = 1 )
        #df.drop('open',inplace = True, axis = 1)

        blocks = list()
        return_Df= df[:-n]
        return_Df.reset_index(inplace= True)
        return_Df.drop('index',inplace = True, axis = 1)
        for i in range(1,n):
            blocks.append(df[i:-(n-i)])
            blocks[i-1].reset_index(inplace= True)
            blocks[i-1].drop('index',inplace = True, axis = 1)
            blocks[i-1].rename(columns={"open":"open " + str(i),"high": "high "+ str(i), "low": "low "+ str(i), "close": "close "+ str(i)},inplace = True)
            return_Df = return_Df.join(blocks[i-1])
        return_Df = return_Df.drop(return_Df.tail(1).index)#?

        view =df['close'][n+1:]

        view.reset_index(inplace=True, drop=True)
        #index = ()
        #print(view > return_Df['close'])
        #print(view.index.difference(index))
        index1 = view > (return_Df['close '+str(n-1)]*percent)

        index2 = view <= (return_Df['close '+str(n-1)]*percent)

        view[index1]='Up' 
        view[index2]='Down' 
        view.rename("Tag",inplace = True)

        return_Df = return_Df.join(view)


        return return_Df
b=  Create_Train(df,15,1.0005)  

/home/noat/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/noat/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [557]:
b.isna().sum()

open        0
high        0
low         0
close       0
open 1      0
           ..
open 14     0
high 14     0
low 14      0
close 14    0
Tag         0
Length: 61, dtype: int64

In [561]:
def Create_Train_candel(df,n):# high - open, open - close, close - low 
    return_df = pd.concat([(df["high"]- df["open"]).rename("high"),(df["open"]- df["close"]).rename("now"),(df["close"] - df["low"]).rename("low")],axis=1)
    for i in range(1,n):
        return_df = return_df.join((df["high "+ str(i)]- df["open "+str(i)]).rename("high "+str(i)))
        return_df = return_df.join((df["open "+ str(i)] - df["close " +str(i)]).rename("now "+str(i)))
        return_df = return_df.join((df["close "+ str(i)] - df["low "+str(i)]).rename("low "+str(i)))
    return return_df.join(b["Tag"])
test =Create_Train_candel(b,15)

In [96]:
def Percent_df(df,n):
        # df.drop('timestamp', inplace = True, axis = 1 )
        # df.drop('close_time', inplace = True, axis = 1 )
        # df.drop('quote_av', inplace = True, axis = 1 )
        # df.drop('trades', inplace = True, axis = 1 )
        # df.drop('tb_base_av', inplace = True, axis = 1 )
        # df.drop('tb_quote_av', inplace = True, axis = 1 )
        # df.drop('ignore', inplace = True, axis = 1 )
        # df.drop('volume', inplace = True, axis = 1 )
        #df.drop('open',inplace = True, axis = 1)

        blocks = list()
        return_Df= df[:-n]
        return_Df.reset_index(inplace= True)
        return_Df.drop('index',inplace = True, axis = 1)
        for i in range(1,n):
            blocks.append(df[i:-(n-i)])
            blocks[i-1].reset_index(inplace= True)
            blocks[i-1].drop('index',inplace = True, axis = 1)
            blocks[i-1].rename(columns={"open":"open " + str(i),"high": "high "+ str(i), "low": "low "+ str(i), "close": "close "+ str(i)},inplace = True)
            return_Df = return_Df.join(blocks[i-1])
        return_Df.drop(return_Df.tail(1).index,inplace=True)

        view =df['close'][n+1:]

        view.reset_index(inplace=True, drop=True)
        #index = ()
        #print(view > return_Df['close'])
        #print(view.index.difference(index))
        view = (view /(return_Df['close 7'])-1)*100

        # view[index1]='Up' 
        # view[index2]='Down' 
        view.rename("Tag",inplace = True)

        return_Df = return_Df.join(view)


        return return_Df
c=  Percent_df(df,8)  

/home/noat/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/noat/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [562]:
test.isna().sum()

high       0
now        0
low        0
high 1     0
now 1      0
low 1      0
high 2     0
now 2      0
low 2      0
high 3     0
now 3      0
low 3      0
high 4     0
now 4      0
low 4      0
high 5     0
now 5      0
low 5      0
high 6     0
now 6      0
low 6      0
high 7     0
now 7      0
low 7      0
high 8     0
now 8      0
low 8      0
high 9     0
now 9      0
low 9      0
high 10    0
now 10     0
low 10     0
high 11    0
now 11     0
low 11     0
high 12    0
now 12     0
low 12     0
high 13    0
now 13     0
low 13     0
high 14    0
now 14     0
low 14     0
Tag        0
dtype: int64

In [563]:
test

high     now     low  high 1   now 1   low 1  high 2   now 2   low 2  \
0      22.20  -10.62   67.24    2.58   71.58   25.57   75.68  -71.92  101.62   
1       2.58   71.58   25.57   75.68  -71.92  101.62   10.10    1.86   26.28   
2      75.68  -71.92  101.62   10.10    1.86   26.28   45.40  -19.67   21.96   
3      10.10    1.86   26.28   45.40  -19.67   21.96   10.52    5.33   16.97   
4      45.40  -19.67   21.96   10.52    5.33   16.97   15.18   35.68    1.11   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
64876  89.56  -27.43  103.16   15.12   38.79  122.16   83.84   41.08   70.23   
64877  15.12   38.79  122.16   83.84   41.08   70.23    6.30  166.97  181.40   
64878  83.84   41.08   70.23    6.30  166.97  181.40   36.36  -20.49  134.89   
64879   6.30  166.97  181.40   36.36  -20.49  134.89  100.51  -10.12  102.25   
64880  36.36  -20.49  134.89  100.51  -10.12  102.25  176.29  -59.25  156.43   

       high 3  ...  high 12  now 12  low 12  high 13  now 13  low 13  high 14  \
0       10.10  ...    14.70   13.70   17.06    24.06    5.74   18.93    12.58   
1       45.40  ...    24.06    5.74   18.93    12.58    5.85    2.34    34.11   
2       10.52  ...    12.58    5.85    2.34    34.11  -33.40   35.14    22.07   
3       15.18  ...    34.11  -33.40   35.14    22.07    3.53    7.52     3.66   
4       15.48  ...    22.07    3.53    7.52     3.66   16.80    4.80    20.40   
...       ...  ...      ...     ...     ...      ...     ...     ...      ...   
64876    6.30  ...    31.49  398.69   83.35   264.92 -134.52  139.93   271.05   
64877   36.36  ...   264.92 -134.52  139.93   271.05 -185.07  190.34     6.00   
64878  100.51  ...   271.05 -185.07  190.34     6.00   39.82  104.84    54.20   
64879  176.29  ...     6.00   39.82  104.84    54.20  -33.44  139.46   222.82   
64880  117.41  ...    54.20  -33.44  139.46   222.82 -216.23  319.88   127.67   

       now 14  low 14   Tag  
0        5.85    2.34    Up  
1      -33.40   35.14  Down  
2        3.53    7.52  Down  
3       16.80    4.80    Up  
4      -12.11   19.93  Down  
...       ...     ...   ...  
64876 -185.07  190.34  Down  
64877   39.82  104.84    Up  
64878  -33.44  139.46    Up  
64879 -216.23  319.88  Down  
64880   43.83  172.35  Down  

[64881 rows x 46 columns]

In [564]:
b= test

In [565]:
b

high     now     low  high 1   now 1   low 1  high 2   now 2   low 2  \
0      22.20  -10.62   67.24    2.58   71.58   25.57   75.68  -71.92  101.62   
1       2.58   71.58   25.57   75.68  -71.92  101.62   10.10    1.86   26.28   
2      75.68  -71.92  101.62   10.10    1.86   26.28   45.40  -19.67   21.96   
3      10.10    1.86   26.28   45.40  -19.67   21.96   10.52    5.33   16.97   
4      45.40  -19.67   21.96   10.52    5.33   16.97   15.18   35.68    1.11   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
64876  89.56  -27.43  103.16   15.12   38.79  122.16   83.84   41.08   70.23   
64877  15.12   38.79  122.16   83.84   41.08   70.23    6.30  166.97  181.40   
64878  83.84   41.08   70.23    6.30  166.97  181.40   36.36  -20.49  134.89   
64879   6.30  166.97  181.40   36.36  -20.49  134.89  100.51  -10.12  102.25   
64880  36.36  -20.49  134.89  100.51  -10.12  102.25  176.29  -59.25  156.43   

       high 3  ...  high 12  now 12  low 12  high 13  now 13  low 13  high 14  \
0       10.10  ...    14.70   13.70   17.06    24.06    5.74   18.93    12.58   
1       45.40  ...    24.06    5.74   18.93    12.58    5.85    2.34    34.11   
2       10.52  ...    12.58    5.85    2.34    34.11  -33.40   35.14    22.07   
3       15.18  ...    34.11  -33.40   35.14    22.07    3.53    7.52     3.66   
4       15.48  ...    22.07    3.53    7.52     3.66   16.80    4.80    20.40   
...       ...  ...      ...     ...     ...      ...     ...     ...      ...   
64876    6.30  ...    31.49  398.69   83.35   264.92 -134.52  139.93   271.05   
64877   36.36  ...   264.92 -134.52  139.93   271.05 -185.07  190.34     6.00   
64878  100.51  ...   271.05 -185.07  190.34     6.00   39.82  104.84    54.20   
64879  176.29  ...     6.00   39.82  104.84    54.20  -33.44  139.46   222.82   
64880  117.41  ...    54.20  -33.44  139.46   222.82 -216.23  319.88   127.67   

       now 14  low 14   Tag  
0        5.85    2.34    Up  
1      -33.40   35.14  Down  
2        3.53    7.52  Down  
3       16.80    4.80    Up  
4      -12.11   19.93  Down  
...       ...     ...   ...  
64876 -185.07  190.34  Down  
64877   39.82  104.84    Up  
64878  -33.44  139.46    Up  
64879 -216.23  319.88  Down  
64880   43.83  172.35  Down  

[64881 rows x 46 columns]

In [566]:
b['Tag'].value_counts()

Down    36485
Up      28396
Name: Tag, dtype: int64

index add weight


In [580]:
#arr=
#b = b.drop(b[b.max(axis=1)==0].index)

#Tag = b['Tag']

#CHIA MAX SCALER
#arr = b.drop('Tag',axis=1).div(b.max(axis=1),axis=0)

#arr = arr.join(Tag)
arr=b

In [581]:
arr

high     now     low  high 1   now 1   low 1  high 2   now 2   low 2  \
0      22.20  -10.62   67.24    2.58   71.58   25.57   75.68  -71.92  101.62   
1       2.58   71.58   25.57   75.68  -71.92  101.62   10.10    1.86   26.28   
2      75.68  -71.92  101.62   10.10    1.86   26.28   45.40  -19.67   21.96   
3      10.10    1.86   26.28   45.40  -19.67   21.96   10.52    5.33   16.97   
4      45.40  -19.67   21.96   10.52    5.33   16.97   15.18   35.68    1.11   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
64876  89.56  -27.43  103.16   15.12   38.79  122.16   83.84   41.08   70.23   
64877  15.12   38.79  122.16   83.84   41.08   70.23    6.30  166.97  181.40   
64878  83.84   41.08   70.23    6.30  166.97  181.40   36.36  -20.49  134.89   
64879   6.30  166.97  181.40   36.36  -20.49  134.89  100.51  -10.12  102.25   
64880  36.36  -20.49  134.89  100.51  -10.12  102.25  176.29  -59.25  156.43   

       high 3  ...  high 12  now 12  low 12  high 13  now 13  low 13  high 14  \
0       10.10  ...    14.70   13.70   17.06    24.06    5.74   18.93    12.58   
1       45.40  ...    24.06    5.74   18.93    12.58    5.85    2.34    34.11   
2       10.52  ...    12.58    5.85    2.34    34.11  -33.40   35.14    22.07   
3       15.18  ...    34.11  -33.40   35.14    22.07    3.53    7.52     3.66   
4       15.48  ...    22.07    3.53    7.52     3.66   16.80    4.80    20.40   
...       ...  ...      ...     ...     ...      ...     ...     ...      ...   
64876    6.30  ...    31.49  398.69   83.35   264.92 -134.52  139.93   271.05   
64877   36.36  ...   264.92 -134.52  139.93   271.05 -185.07  190.34     6.00   
64878  100.51  ...   271.05 -185.07  190.34     6.00   39.82  104.84    54.20   
64879  176.29  ...     6.00   39.82  104.84    54.20  -33.44  139.46   222.82   
64880  117.41  ...    54.20  -33.44  139.46   222.82 -216.23  319.88   127.67   

       now 14  low 14   Tag  
0        5.85    2.34    Up  
1      -33.40   35.14  Down  
2        3.53    7.52  Down  
3       16.80    4.80    Up  
4      -12.11   19.93  Down  
...       ...     ...   ...  
64876 -185.07  190.34  Down  
64877   39.82  104.84    Up  
64878  -33.44  139.46    Up  
64879 -216.23  319.88  Down  
64880   43.83  172.35  Down  

[64881 rows x 46 columns]

high     now     low  high 1   now 1   low 1  high 2   now 2   low 2  \
0       22.20  -10.62   67.24    2.58   71.58   25.57   75.68  -71.92  101.62   
1        2.58   71.58   25.57   75.68  -71.92  101.62   10.10    1.86   26.28   
2       75.68  -71.92  101.62   10.10    1.86   26.28   45.40  -19.67   21.96   
3       10.10    1.86   26.28   45.40  -19.67   21.96   10.52    5.33   16.97   
4       45.40  -19.67   21.96   10.52    5.33   16.97   15.18   35.68    1.11   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
64888   31.49  398.69   83.35  264.92 -134.52  139.93  271.05 -185.07  190.34   
64889  264.92 -134.52  139.93  271.05 -185.07  190.34    6.00   39.82  104.84   
64890  271.05 -185.07  190.34    6.00   39.82  104.84   54.20  -33.44  139.46   
64891    6.00   39.82  104.84   54.20  -33.44  139.46  222.82 -216.23  319.88   
64892   54.20  -33.44  139.46  222.82 -216.23  319.88  127.67   43.83  172.35   

        Tag  
0        Up  
1        Up  
2      Down  
3      Down  
4      Down  
...     ...  
64888  Down  
64889    Up  
64890    Up  
64891  Down  
64892  Down  

[64887 rows x 10 columns]

In [582]:
(b.max(axis=1)==0).sum()

0

In [583]:
b.drop('Tag',axis=1).div(b.max(axis=1),axis=0).isna().sum()

high       0
now        0
low        0
high 1     0
now 1      0
low 1      0
high 2     0
now 2      0
low 2      0
high 3     0
now 3      0
low 3      0
high 4     0
now 4      0
low 4      0
high 5     0
now 5      0
low 5      0
high 6     0
now 6      0
low 6      0
high 7     0
now 7      0
low 7      0
high 8     0
now 8      0
low 8      0
high 9     0
now 9      0
low 9      0
high 10    0
now 10     0
low 10     0
high 11    0
now 11     0
low 11     0
high 12    0
now 12     0
low 12     0
high 13    0
now 13     0
low 13     0
high 14    0
now 14     0
low 14     0
dtype: int64

SyntaxError: invalid syntax (<ipython-input-448-618920615d00>, line 1)

In [584]:
y_sr = arr["Tag"] 
X_df = arr.drop("Tag", axis=1)

In [585]:
train_X_df.isna().sum()

high       0
now        0
low        0
high 1     0
now 1      0
low 1      0
high 2     0
now 2      0
low 2      0
high 3     0
now 3      0
low 3      0
high 4     0
now 4      0
low 4      0
high 5     0
now 5      0
low 5      0
high 6     0
now 6      0
low 6      0
high 7     0
now 7      0
low 7      0
high 8     0
now 8      0
low 8      0
high 9     0
now 9      0
low 9      0
high 10    0
now 10     0
low 10     0
high 11    0
now 11     0
low 11     0
high 12    0
now 12     0
low 12     0
high 13    0
now 13     0
low 13     0
high 14    0
now 14     0
low 14     0
dtype: int64

In [586]:

train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)

In [587]:
mlpc = MLPClassifier(hidden_layer_sizes=(100),random_state=0, max_iter=2500)

In [593]:
full_pipeline = make_pipeline(MinMaxScaler(),mlpc)


In [594]:
full_pipeline.fit(train_X_df, train_y_sr)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('mlpclassifier',
                 MLPClassifier(hidden_layer_sizes=100, max_iter=2500,
                               random_state=0))])

In [595]:
full_pipeline.score(val_X_df , val_y_sr )

0.5642948882609813

In [591]:
full_pipeline.fit(val_X_df , val_y_sr )

Pipeline(steps=[('mlpclassifier',
                 MLPClassifier(hidden_layer_sizes=100, max_iter=2500,
                               random_state=0))])

In [592]:
full_pipeline.score(train_X_df, train_y_sr)

0.5617843931654043

# Làm quả hàm chuẩn bị dữ lịu tét

In [365]:
b

high     now     low  high 1   now 1   low 1  high 2   now 2   low 2  \
0       22.20  -10.62   67.24    2.58   71.58   25.57   75.68  -71.92  101.62   
1        2.58   71.58   25.57   75.68  -71.92  101.62   10.10    1.86   26.28   
2       75.68  -71.92  101.62   10.10    1.86   26.28   45.40  -19.67   21.96   
3       10.10    1.86   26.28   45.40  -19.67   21.96   10.52    5.33   16.97   
4       45.40  -19.67   21.96   10.52    5.33   16.97   15.18   35.68    1.11   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
64883  117.41  -50.09  192.22  727.65 -712.66  771.97  161.26 -111.91  222.54   
64884  727.65 -712.66  771.97  161.26 -111.91  222.54  346.09 -251.57  251.58   
64885  161.26 -111.91  222.54  346.09 -251.57  251.58  209.52   18.45  118.68   
64886  346.09 -251.57  251.58  209.52   18.45  118.68   31.49  398.69   83.35   
64887  209.52   18.45  118.68   31.49  398.69   83.35  264.92 -134.52  139.93   

       high 3  ...  high 5   now 5   low 5  high 6   now 6   low 6  high 7  \
0       10.10  ...   10.52    5.33   16.97   15.18   35.68    1.11   15.48   
1       45.40  ...   15.18   35.68    1.11   15.48   -3.89   52.75   18.99   
2       10.52  ...   15.48   -3.89   52.75   18.99   20.12    3.86   54.13   
3       15.18  ...   18.99   20.12    3.86   54.13  -52.46   52.48   37.47   
4       15.48  ...   54.13  -52.46   52.48   37.47  -24.15   33.68    6.42   
...       ...  ...     ...     ...     ...     ...     ...     ...     ...   
64883  346.09  ...   31.49  398.69   83.35  264.92 -134.52  139.93  271.05   
64884  209.52  ...  264.92 -134.52  139.93  271.05 -185.07  190.34    6.00   
64885   31.49  ...  271.05 -185.07  190.34    6.00   39.82  104.84   54.20   
64886  264.92  ...    6.00   39.82  104.84   54.20  -33.44  139.46  222.82   
64887  271.05  ...   54.20  -33.44  139.46  222.82 -216.23  319.88  127.67   

        now 7   low 7   Tag  
0       -3.89   52.75    Up  
1       20.12    3.86    Up  
2      -52.46   52.48    Up  
3      -24.15   33.68  Down  
4       13.56    4.08  Down  
...       ...     ...   ...  
64883 -185.07  190.34  Down  
64884   39.82  104.84    Up  
64885  -33.44  139.46    Up  
64886 -216.23  319.88  Down  
64887   43.83  172.35  Down  

[64888 rows x 25 columns]

In [250]:
full_pipeline.predict(val_X_df.tail(10))

array(['Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down', 'Down',
       'Down', 'Down'], dtype='<U4')

In [327]:
full_pipeline.predict_proba(val_X_df.tail(10))

array([[0.51057224, 0.48942776],
       [0.54235313, 0.45764687],
       [0.61968891, 0.38031109],
       [0.60422429, 0.39577571],
       [0.52354495, 0.47645505],
       [0.52658549, 0.47341451],
       [0.59218376, 0.40781624],
       [0.61095898, 0.38904102],
       [0.64902369, 0.35097631],
       [0.60748307, 0.39251693]])

In [328]:
val_y_sr.tail(10)

18646    Down
38137    Down
41685    Down
51575    Down
23936    Down
22514    Down
60085    Down
2407     Down
8362       Up
2375       Up
Name: Tag, dtype: object

In [329]:
pro_arr = full_pipeline.predict_proba(val_X_df)

In [332]:
np.sum((pro_arr[:,1]>0.6))

98

In [204]:
from binance.client import Client
import datetime

# YOUR API KEYS HERE
#api_key = "1xkfS28jR7eW8EZk77BCt3NtznzgIoJXjxTelowoDPD8MXvBUfBazJV0CSzF0Jwq"
#api_secret = "iTqdOQHe32KQvhG33yC68bk5wTKRkb2zlszwM9x63sXyMjwxAYe6WerSj5SW4g5l"

api_key = "ANUulMmh5ClMUu4lFgoahqBoqgIFCPGAlz2vIwHdjk80HQSMp9miHKwECPdiCAVJ"    #Enter your own API-key here
api_secret = "mv8RXmGzA4dLC5wgXvb4Op14otLUNm3uEU9egCxRYX2h3S2Ns7mrnHifg7Hu8gQS" #Enter your own API-secret here
bclient = Client(api_key=api_key, api_secret=api_secret)

start_date = datetime.datetime.strptime('23 May 2019', '%d %b %Y')
today = datetime.datetime.today()

def binanceBarExtractor(symbol):
    print('working...')
    filename = '{}_100MinuteBars.csv'.format(symbol)

    klines = bclient.get_historical_klines(symbol, Client.KLINE_INTERVAL_15MINUTE, start_date.strftime("%d %b %Y %H:%M:%S"), today.strftime("%d %b %Y %H:%M:%S"), 1000)
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')

    data.set_index('timestamp', inplace=True)
    data.to_csv(filename)
    print('finished!')


if __name__ == '__main__':
    # Obviously replace BTCUSDT with whichever symbol you want from binance
    # Wherever you've saved this code is the same directory you will find the resulting CSV file
    #binanceBarExtractor('BTCUSDT')
    balance = bclient.get_asset_balance(asset='REEF')
    print(balance)


BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action.

In [558]:
df2 = pd.read_csv('BTCUSDT_30MinuteBars_by_Now.csv ')
df2

timestamp      open      high       low     close  \
0     2021-02-23 00:00:00  54087.67  54183.59  53018.54  53345.33   
1     2021-02-23 00:30:00  53345.33  53892.42  53345.33  53440.66   
2     2021-02-23 01:00:00  53431.44  53925.86  52632.58  52756.44   
3     2021-02-23 01:30:00  52753.44  53094.67  52281.88  52546.18   
4     2021-02-23 02:00:00  52546.59  52887.66  51919.53  51993.13   
...                   ...       ...       ...       ...       ...   
1363  2021-03-23 11:00:00  54671.05  54804.98  54540.00  54688.95   
1364  2021-03-23 11:30:00  54688.95  54783.76  54060.79  54249.81   
1365  2021-03-23 12:00:00  54249.80  55250.00  54234.96  55112.70   
1366  2021-03-23 12:30:00  55108.86  55287.52  54915.05  55110.00   
1367  2021-03-23 13:00:00  55110.00  55352.00  55061.17  55263.05   

           volume     close_time      quote_av  trades   tb_base_av  \
0     1831.952862  1614040199999  9.792048e+07   60528   907.604301   
1     1111.416103  1614041999999  5.961173e+07   46849   556.065013   
2     1436.512693  1614043799999  7.643267e+07   40358   684.077372   
3     1696.081723  1614045599999  8.939923e+07   43041   887.152925   
4     2468.505922  1614047399999  1.292997e+08   68279  1292.698071   
...           ...            ...           ...     ...          ...   
1363   731.492546  1616498999999  4.000558e+07   24553   383.708113   
1364  1465.468429  1616500799999  7.972295e+07   41084   723.387430   
1365  1936.556851  1616502599999  1.062491e+08   52551  1066.828563   
1366   999.450700  1616504399999  5.502668e+07   30343   510.115722   
1367   976.648776  1616506199999  5.391707e+07   27920   452.824877   

       tb_quote_av  ignore  
0     4.850729e+07       0  
1     2.982769e+07       0  
2     3.639869e+07       0  
3     4.676555e+07       0  
4     6.771472e+07       0  
...            ...     ...  
1363  2.098597e+07       0  
1364  3.934464e+07       0  
1365  5.851459e+07       0  
1366  2.808689e+07       0  
1367  2.500011e+07       0  

[1368 rows x 12 columns]

In [559]:
data = Create_Train(df2,8)

C:\Users\Toan\anaconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Toan\anaconda3\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [560]:
data.tail(1)

high       low     close    high 1     low 1   close 1   high 2  \
1358  54482.81  53916.34  54309.24  54485.06  54215.24  54313.62  54698.3   

        low 2   close 2   high 3  ...    high 5    low 5   close 5    high 6  \
1358  54230.5  54607.86  54723.0  ...  54804.98  54540.0  54688.95  54783.76   

         low 6   close 6   high 7     low 7  close 7  Tag  
1358  54060.79  54249.81  55250.0  54234.96  55112.7   Up  

[1 rows x 25 columns]

In [564]:
full_pipeline.predict(data.tail(1).drop('Tag',axis=1))

array(['Up'], dtype='<U4')